# Package import

In [110]:
import json
import requests
import ndjson
import pandas as pd
import os
from pandas_profiling import ProfileReport

# Functions

In [111]:
import ndjson
from typing import List, Dict

def load_ndjon(file_path: str) -> List[Dict]:
    with open(file_path) as f:
        return ndjson.load(f)

# Testing stuff

In [112]:


url = "https://api.openaq.org/v2/sources?limit=100&page=1&offset=0&sort=asc&order_by=sourceName"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

json_data = response.json()
json_data['results'][0]


{'data': {'url': 'https://www.adairquality.ae/',
  'data_avg_dur': None,
  'organization': None,
  'lifecycle_stage': None},
 'readme': None,
 'sourceId': 9800851,
 'locations': 81,
 'sourceName': 'Abu Dhabi Air Quality',
 'sourceSlug': None}

# Exploring the data

In [113]:
countries_df = pd.read_csv('../raw_data/countries.csv')
countries_df

,country_code,country_name
0,GB,Great Britain
1,FR,France
2,NL,Netherlands


In [114]:
data_path = os.path.join('..', 'raw_data', 'air_quality')

data = []
for filename in os.listdir(data_path):
    data.extend(load_ndjon(os.path.join(data_path, filename)))
    


    
first_element = data[0]
first_element

{'date': {'utc': '2021-08-15T14:30:00.000Z',
  'local': '2021-08-15T19:00:00+04:30'},
 'parameter': 'pm25',
 'value': -999,
 'unit': 'µg/m³',
 'averagingPeriod': {'value': 1, 'unit': 'hours'},
 'location': 'US Diplomatic Post: Kabul',
 'city': 'Kabul',
 'country': 'AF',
 'coordinates': {'latitude': 34.535812, 'longitude': 69.190514},
 'attribution': [{'name': 'EPA AirNow DOS',
   'url': 'http://airnow.gov/index.cfm?action=airnow.global_summary'}],
 'sourceName': 'StateAir_Kabul',
 'sourceType': 'government',
 'mobile': False}

In [115]:
df = pd.json_normalize(data, sep="_")

df.head()

,parameter,value,unit,location,city,country,attribution,sourceName,sourceType,mobile,date_utc,date_local,averagingPeriod_value,averagingPeriod_unit,coordinates_latitude,coordinates_longitude
0,pm25,-999.0,µg/m³,US Diplomatic Post: Kabul,Kabul,AF,"[{'name': 'EPA AirNow DOS', 'url': 'http://air...",StateAir_Kabul,government,False,2021-08-15T14:30:00.000Z,2021-08-15T19:00:00+04:30,1.0,hours,34.535812,69.190514
1,pm25,-999.0,µg/m³,US Diplomatic Post: Kabul,Kabul,AF,"[{'name': 'EPA AirNow DOS', 'url': 'http://air...",StateAir_Kabul,government,False,2021-08-15T15:30:00.000Z,2021-08-15T20:00:00+04:30,1.0,hours,34.535812,69.190514
2,pm25,-999.0,µg/m³,US Diplomatic Post: Kabul,Kabul,AF,"[{'name': 'EPA AirNow DOS', 'url': 'http://air...",StateAir_Kabul,government,False,2021-08-15T16:30:00.000Z,2021-08-15T21:00:00+04:30,1.0,hours,34.535812,69.190514
3,pm25,-999.0,µg/m³,US Diplomatic Post: Kabul,Kabul,AF,"[{'name': 'EPA AirNow DOS', 'url': 'http://air...",StateAir_Kabul,government,False,2021-08-15T17:30:00.000Z,2021-08-15T22:00:00+04:30,1.0,hours,34.535812,69.190514
4,pm25,-999.0,µg/m³,US Diplomatic Post: Kabul,Kabul,AF,"[{'name': 'EPA AirNow DOS', 'url': 'http://air...",StateAir_Kabul,government,False,2021-08-15T18:30:00.000Z,2021-08-15T23:00:00+04:30,1.0,hours,34.535812,69.190514


In [116]:
df.dtypes

parameter                 object
value                    float64
unit                      object
location                  object
city                      object
country                   object
attribution               object
sourceName                object
sourceType                object
mobile                      bool
date_utc                  object
date_local                object
averagingPeriod_value    float64
averagingPeriod_unit      object
coordinates_latitude     float64
coordinates_longitude    float64
dtype: object

In [117]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77273 entries, 0 to 77272
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   parameter              77273 non-null  object 
 1   value                  77273 non-null  float64
 2   unit                   77273 non-null  object 
 3   location               77273 non-null  object 
 4   city                   77273 non-null  object 
 5   country                77273 non-null  object 
 6   attribution            77273 non-null  object 
 7   sourceName             77273 non-null  object 
 8   sourceType             77273 non-null  object 
 9   mobile                 77273 non-null  bool   
 10  date_utc               77273 non-null  object 
 11  date_local             77273 non-null  object 
 12  averagingPeriod_value  77273 non-null  float64
 13  averagingPeriod_unit   77273 non-null  object 
 14  coordinates_latitude   77063 non-null  float64
 15  co

In [118]:
profile = ProfileReport(df, title="Air Quality data Report")
profile.to_file("air_quality_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]/home/alkis/.pyenv/versions/3.10.7/envs/openAQ/lib/python3.10/site-packages/multimethod/__init__.py:315: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  return func(*args, **kwargs)
Summarize dataset:  61%|██████    | 17/28 [00:03<00:01,  6.41it/s, Calculate auto correlation]             /home/alkis/.pyenv/versions/3.10.7/envs/openAQ/lib/python3.10/site-packages/multimethod/__init__.py:315: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  return func(*args, **kwargs)
Summarize dataset:  64%|██████▍   | 18/28 [00:05<00:02,  4.96it/s, Calculate spearman correlation]/home/alkis/.pyenv/versions/3.10.7/envs/openAQ/lib/python3.10/site-packages/multimethod/__init__.py:3

# Filtering the data

In [119]:
from enum import Enum, unique

@unique
class AirQualityIndex(Enum):
    PM25: str = 'pm25'
    PM10: str = 'pm10'
    O3: str = 'o3'
    NO2: str = 'no2'
    CO: str = 'co'
    
    @classmethod
    def to_list(cls):
        return [index.value  for index in cls.__members__.values()]
        
    

AirQualityIndex.to_list()
    

['pm25', 'pm10', 'o3', 'no2', 'co']

In [120]:



counties = list(countries_df.country_code)
paramaters = AirQualityIndex.to_list()

def filter_air_quality_data(df: pd.DataFrame, countries: List[str], paramaters: List[str]) -> pd.DataFrame:
    # columns_filter = ['parameter', 'value', 'city','date_utc', 'date_local', 'averagingPeriod_value', 'averagingPeriod_unit']
    filter = df['country'].isin(countries) & df['parameter'].isin(paramaters)
    return df[filter].copy()


def filter_by_parameter(df, parameter):
    return df[df['parameter'] == parameter].copy()



filtered_df = filter_air_quality_data(df, counties, paramaters)
filtered_df['date_utc'] = pd.to_datetime(filtered_df['date_utc'])
filtered_df['date_local'] = pd.to_datetime(filtered_df['date_local'])

# filtered_df['date_utc'] = pd.to_datetime(filtered_df['date_utc'])
pm25_df = filter_by_parameter(filtered_df, AirQualityIndex.PM25.value)





In [121]:
pm25_df.city.unique()

array(['Auchencorth', 'Edinburgh', 'Glasgow', 'Grangemouth',
       'Central Scotland', 'Chesterfield', 'Leicester', 'Nottingham',
       'Norwich', 'Sandy', 'London', 'Stanford-le-Hope', 'Inverness',
       'Middlesbrough', 'Newcastle', 'Stockton-on-Tees', 'Sunderland',
       'Wrexham', 'Blackpool', 'Manchester', 'Preston', 'Warrington',
       'Wigan', 'Liverpool', 'Belfast', 'Derry', 'Lough Navar', 'Chatham',
       'Stockbridge', 'Eastbourne', 'Oxford', 'Reading', 'Rochester',
       'Southampton', 'South East', 'Cardiff', 'Chepstow', 'Narberth',
       'Newport', 'Port Talbot', 'Swansea', 'Barnstaple', 'South West',
       'Plymouth', 'Saltash', 'West Midlands', 'Coventry',
       'Leamington Spa', 'Stoke-on-Trent', 'Hull', 'Leeds',
       'Yorkshire & Humberside', 'Sheffield', 'York', 'Bexley', 'Brent',
       'City of London', 'Crawley', 'Croydon', 'Greenwich', 'Lewisham',
       'Richmond', 'Newham', 'N/A', 'Doubs', 'Bouches-du-Rhône',
       'Loir-et-Cher', 'Eure-et-Loir', 'N

In [122]:
pm25_df[pm25_df['city'] == 'Amsterdam']

,parameter,value,unit,location,city,country,attribution,sourceName,sourceType,mobile,date_utc,date_local,averagingPeriod_value,averagingPeriod_unit,coordinates_latitude,coordinates_longitude
50219,pm25,1.4,µg/m³,Amsterdam-Einsteinweg,Amsterdam,NL,"[{'name': 'RIVM', 'url': 'http://www.lml.rivm....",Netherlands,government,False,2021-10-06 14:00:00+00:00,2021-10-06 16:00:00+02:00,24.0,hours,NaN,NaN
50220,pm25,5.8,µg/m³,Amsterdam-Van Diemenstraat,Amsterdam,NL,"[{'name': 'RIVM', 'url': 'http://www.lml.rivm....",Netherlands,government,False,2021-10-06 14:00:00+00:00,2021-10-06 16:00:00+02:00,24.0,hours,NaN,NaN
50221,pm25,2.1,µg/m³,Amsterdam-Vondelpark,Amsterdam,NL,"[{'name': 'RIVM', 'url': 'http://www.lml.rivm....",Netherlands,government,False,2021-10-06 14:00:00+00:00,2021-10-06 16:00:00+02:00,24.0,hours,NaN,NaN
50222,pm25,0.5,µg/m³,Amsterdam-Westerpark,Amsterdam,NL,"[{'name': 'RIVM', 'url': 'http://www.lml.rivm....",Netherlands,government,False,2021-10-06 14:00:00+00:00,2021-10-06 16:00:00+02:00,24.0,hours,NaN,NaN
50223,pm25,9.7,µg/m³,Amsterdam-Stadhouderskade,Amsterdam,NL,"[{'name': 'RIVM', 'url': 'http://www.lml.rivm....",Netherlands,government,False,2021-10-06 14:00:00+00:00,2021-10-06 16:00:00+02:00,24.0,hours,NaN,NaN
50231,pm25,3.3,µg/m³,Amsterdam-Spaarnwoude,Amsterdam,NL,"[{'name': 'RIVM', 'url': 'http://www.lml.rivm....",Netherlands,government,False,2021-10-06 14:00:00+00:00,2021-10-06 16:00:00+02:00,24.0,hours,NaN,NaN
50232,pm25,4.2,µg/m³,Amsterdam-Hoogtij,Amsterdam,NL,"[{'name': 'RIVM', 'url': 'http://www.lml.rivm....",Netherlands,government,False,2021-10-06 14:00:00+00:00,2021-10-06 16:00:00+02:00,24.0,hours,NaN,NaN
63844,pm25,5.1,µg/m³,Amsterdam-Einsteinweg,Amsterdam,NL,"[{'name': 'RIVM', 'url': 'http://www.lml.rivm....",Netherlands,government,False,2021-10-06 17:00:00+00:00,2021-10-06 19:00:00+02:00,24.0,hours,NaN,NaN
63845,pm25,7.8,µg/m³,Amsterdam-Van Diemenstraat,Amsterdam,NL,"[{'name': 'RIVM', 'url': 'http://www.lml.rivm....",Netherlands,government,False,2021-10-06 17:00:00+00:00,2021-10-06 19:00:00+02:00,24.0,hours,NaN,NaN
63846,pm25,6.0,µg/m³,Amsterdam-Vondelpark,Amsterdam,NL,"[{'name': 'RIVM', 'url': 'http://www.lml.rivm....",Netherlands,government,False,2021-10-06 17:00:00+00:00,2021-10-06 19:00:00+02:00,24.0,hours,NaN,NaN


In [149]:
pm25_24h = pm25_df[(pm25_df['averagingPeriod_value'] == 24)]
pm25_24h[pm25_24h['city'] == 'Glasgow']
# pm25_24h

,parameter,value,unit,location,city,country,attribution,sourceName,sourceType,mobile,date_utc,date_local,averagingPeriod_value,averagingPeriod_unit,coordinates_latitude,coordinates_longitude
1611,pm25,3.0,µg/m³,Glasgow High Street,Glasgow,GB,[{'name': 'Department for Environmental Food &...,DEFRA,government,False,2021-10-06 16:00:00+00:00,2021-10-06 17:00:00+01:00,24.0,hours,55.860936,-4.238214
1616,pm25,2.0,µg/m³,Glasgow Townhead,Glasgow,GB,[{'name': 'Department for Environmental Food &...,DEFRA,government,False,2021-10-06 16:00:00+00:00,2021-10-06 17:00:00+01:00,24.0,hours,55.865782,-4.243631
15087,pm25,3.0,µg/m³,Glasgow High Street,Glasgow,GB,[{'name': 'Department for Environmental Food &...,DEFRA,government,False,2021-10-06 14:00:00+00:00,2021-10-06 15:00:00+01:00,24.0,hours,55.860936,-4.238214
15092,pm25,2.0,µg/m³,Glasgow Townhead,Glasgow,GB,[{'name': 'Department for Environmental Food &...,DEFRA,government,False,2021-10-06 14:00:00+00:00,2021-10-06 15:00:00+01:00,24.0,hours,55.865782,-4.243631
26656,pm25,3.0,µg/m³,Glasgow High Street,Glasgow,GB,[{'name': 'Department for Environmental Food &...,DEFRA,government,False,2021-10-06 17:00:00+00:00,2021-10-06 18:00:00+01:00,24.0,hours,55.860936,-4.238214
26661,pm25,2.0,µg/m³,Glasgow Townhead,Glasgow,GB,[{'name': 'Department for Environmental Food &...,DEFRA,government,False,2021-10-06 17:00:00+00:00,2021-10-06 18:00:00+01:00,24.0,hours,55.865782,-4.243631
51530,pm25,3.0,µg/m³,Glasgow High Street,Glasgow,GB,[{'name': 'Department for Environmental Food &...,DEFRA,government,False,2021-10-06 18:00:00+00:00,2021-10-06 19:00:00+01:00,24.0,hours,55.860936,-4.238214
51535,pm25,2.0,µg/m³,Glasgow Townhead,Glasgow,GB,[{'name': 'Department for Environmental Food &...,DEFRA,government,False,2021-10-06 18:00:00+00:00,2021-10-06 19:00:00+01:00,24.0,hours,55.865782,-4.243631
65436,pm25,3.0,µg/m³,Glasgow High Street,Glasgow,GB,[{'name': 'Department for Environmental Food &...,DEFRA,government,False,2021-10-06 13:00:00+00:00,2021-10-06 14:00:00+01:00,24.0,hours,55.860936,-4.238214
65441,pm25,2.0,µg/m³,Glasgow Townhead,Glasgow,GB,[{'name': 'Department for Environmental Food &...,DEFRA,government,False,2021-10-06 13:00:00+00:00,2021-10-06 14:00:00+01:00,24.0,hours,55.865782,-4.243631


In [146]:
bexley_df = pm25_df[pm25_df['city'] == 'Bexley']
# bexley_df = bexley_df.set_index('date_utc')
bexley_df.groupby('date_utc')['value'].mean()

date_utc
2021-10-06 00:00:00+00:00    3.900000
2021-10-06 01:00:00+00:00    3.500000
2021-10-06 02:00:00+00:00    3.966667
2021-10-06 03:00:00+00:00    4.266667
2021-10-06 04:00:00+00:00    3.533333
2021-10-06 05:00:00+00:00    3.600000
2021-10-06 06:00:00+00:00    4.366667
2021-10-06 07:00:00+00:00    4.100000
2021-10-06 08:00:00+00:00    5.133333
2021-10-06 09:00:00+00:00    5.366667
2021-10-06 10:00:00+00:00    5.533333
2021-10-06 11:00:00+00:00    4.700000
2021-10-06 12:00:00+00:00    6.687500
2021-10-06 13:00:00+00:00    4.900000
2021-10-06 14:00:00+00:00    5.225000
2021-10-06 15:00:00+00:00    3.300000
2021-10-06 16:00:00+00:00    3.700000
2021-10-06 17:00:00+00:00    5.100000
Name: value, dtype: float64

parameter                             object
value                                float64
unit                                  object
location                              object
city                                  object
country                               object
attribution                           object
sourceName                            object
sourceType                            object
mobile                                  bool
date_utc                 datetime64[ns, UTC]
date_local                            object
averagingPeriod_value                float64
averagingPeriod_unit                  object
coordinates_latitude                 float64
coordinates_longitude                float64
dtype: object

# Data models

In [125]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column
from sqlalchemy.sql.sqltypes import Integer, String

Base = declarative_base()

class DBSources(Base):
    __tablename__="source"
    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String(250), nullable=False)
    sourceId = Column(Integer)
    location = Column(Integer)
    